# Долгосрочное дз.
**ФИО:** Гриднева Е. В.
**Группа:** СКБ222

**Игра:** "Море, соль, бумага"
**Подвариант**: Ракушки-крабы

**Задание:**
```
 1. Реализовать на языке Python вероятностную модель для упрощенных правил одной игры.
 2. Получить данные о проведенных партиях с помощью симуляций.
 3. На основе данных построить оценки и/или проверить гипотезы.

 Для выбранной игры рекомендуется:
 - провести от 10 до 100 тысяч симуляций;
 - полученную выборку использовать для построения оценок и проверки гипотез;
 - если пользоваться бутстрапом, то эта выборка является основной (результаты ваших симуляций), и сэмплировать подвыборки нужно из неё.

Уровень значимости для всех критериев - 0.95
```

## **Правила игры:**
Это карточная игра на сбор сетов (наборов определенных карт). Дуэльный вариант (2 игрока). 

В центре стола находится колода (рубашкой вверх) и 2 стопки сброса (рубашкой вниз, верхняя карта видна).

Игра длится 2 раунда. 

Если разыгранные пары и наборы на руке в сумме приносят 7+ очков, игрок может остановить раунд и объявить подсчет очков. 

Также раунд заканчивается, если в колоде закончились карты.

Выигрывает игрок, набравший больше очков в сумме за два раунда. Игрок, начинавший первый раунд, во втором ходит вторым.

--------------------------------------------------
**В свой ход игрок:**
- либо берет две карты из колоды, одну оставляет, одну сбрасывает в случайный (!) сброс.
- либо берёт открытую карту из любого сброса. 

Затем игрок немедленно разыгрывает карты, составляющие пару и применяет их эффекты. 

Затем, если разыгранные пары и наборы на руке в сумме приносят 7+ очков, игрок может остановить раунд и объявить подсчет очков

--------------------------------------------------
**Очки:**

**Парные карты и их эффекты:**
- 2 рыбы: позволяют взять верхнюю карту колоды;
- 2 корабля: позволяют сделать еще один ход;
- 2 краба: позволяют взять из сброса любую карту;
- Акула + пловец: позволяет взять случайную карту с руки соперника. 

Каждая разыгранная пара после применения лежит перед игроком и приносит 1 очко.

**Наборы из карт:** 

приносят фиксированное количество очков в зависимости от их количества:
- 1/2/3/4/5/6 ракушек приносят 0/2/4/6/8/10 очков
- 1/2/3/4/5 осьминогов приносят 0/3/6/9/12 очков
- 1/2/3 пингвина приносят 1/3/5 очков
- 1/2 моряка приносят 0/5 очков 

--------------------------------------------------
В начале игры колода содержит:
- 9 крабов
- 8 кораблей
- 7 рыб
- 6 ракушек
- 5 акул
- 5 пловцов
- 5 осьминогов
- 3 пингвина
- 2 моряка

## **Задание 1.**
Чтобы корректно провести симуляцию и проверять гипотезы, необходимо выработать стратегию, которая заключается в том, какие карты брать приоритетней (упорядоченный список из всех карт). 

**Моя стратегия:**
- ракушки
- крабы
- рыбы
- акулы
- пингвины 
- осьминоги
- корабли
- моряки
- пловцы

Для всех стратегий будем предполагать, что если топ3 элемента списка не лежат верхней картой, игрок будет испытывать удачу и брать карты из колоды.

У меня, например, игрок всегда будет пытаться взять карту ракушки, если нет, будет пытаться взять краба, а если и её нет, то рыбу. 

Данный порядок приоритетов относится к взятию карту из сброса, выбору из двух карт колоды, а также поиску карт в сбросе с помощью краба.

## **Задание 2.**
**Несколько стратегий-соперников:**

*а) Стратегия, в которой все элементы в обратном порядке:*
- пловцы
- моряки
- корабли
- осьминоги
- пингвины
- акулы
- рыбы
- крабы
- ракушки

*б) Стратегия, в которой парная карта из вашего топ-2 поменяна местами с любой другой парной:*
- ракушки
- корабли
- рыбы
- акулы
- пингвины 
- осьминоги
- крабы
- моряки
- пловцы

*в) Стратегия, в которой непарная карта из вашего топ-2 поменяна местами с любой другой непарной:*
- осьминоги
- крабы
- рыбы
- акулы
- пингвины 
- ракушки
- корабли
- моряки
- пловцы

# **Задание 3.**

Генерирую данные для игр со всеми соперниками для двух вариантов: 

а) любой игрок завершает раунд, как только набрал 7 очков,

б) оба игрока ждут окончания колоды.

In [320]:
import random
from collections import defaultdict
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

✅ Вспомогательные функции

In [321]:
# при подсёте очков смотрим на наборы из карт одной масти
SHELLS = {1: 0, 2: 2, 3: 4, 4: 6, 5: 8, 6: 10}
OCTOPUSES = {1: 0, 2: 3, 3: 6, 4: 9, 5: 12}
PENGUINS = {1: 1, 2: 3, 3: 5}
SAILORS = {1: 0, 2: 5}

SETSPOINTS = [SHELLS, OCTOPUSES, PENGUINS, SAILORS]

NAMES_CARD = ["ракушки", "осьминоги", "пингвины", "моряки"]

# player = {"hand": [], "played_pair": [], "strat": [], "score": [score_pairs, score_sets]}

def resetData(player):
    player["hand"] = []
    player["played_pair"] = []
    player["score"] = [0, 0]



def genDeck():
    deck = (["крабы"] * 9 + ["корабли"] * 8 + 
            ["рыбы"] * 7 + ["ракушки"] * 6 + 
            ["акулы"] * 5 + ["пловцы"] * 5 + 
            ["осьминоги"] * 5 + ["пингвины"] * 3 
            + ["моряки"] * 2)
    
    random.shuffle(deck)
    return deck



def takeCard(deck, sbroses, player):
    card = ""
    top3 = player["strat"][:3]

    card = takeFromSbros(sbroses, top3)
    
    if card == "": # если среди сброса не было приоритетных карт, берем из колоды
        card = takeFromDeck(deck, sbroses, player["strat"])

    if card != "":
        player["hand"].append(card)
        return 1 # успешно взяли карту 
    
    return 0



def takeFromSbros(sbroses, prior):
    idx = random.choice([0, 1])
    first_sbros = sbroses[idx]
    second_sbros = sbroses[1 - idx]

    for p in prior:
        if first_sbros and first_sbros[-1] == p:
            return first_sbros.pop()


        if second_sbros and second_sbros[-1] == p:
            return second_sbros.pop()

    return ""



def takeFromDeck(deck, sbroses, strat):
    if len(deck) >= 2:
        cards = [deck.pop(), deck.pop()]
        idx = random.choice([0, 1])
        card1, card2 = cards[idx], cards[1-idx]

        for s in strat:
            if s == card1:
                random.choice(sbroses).append(card2) # рандомно выбираем сброс в который скидываем невзятую карту
                return card1

            if s == card2:
                random.choice(sbroses).append(card1)
                return card2
    
    elif len(deck) == 1:
        return deck.pop()
    
    return ""



# смотрим на пары -> за каждую новую пару +очко -> применяем эффекты
def playCard(deck, sbroses, player, opponent):
    card = ""

    while True: # пока есть пары
        if player["hand"].count("рыбы") == 2:
            player["score"][0] += 1
            removeFromHand(player, "рыбы", "рыбы")
            if deck: player["hand"].append(deck.pop())

        elif player["hand"].count("корабли") == 2:
            player["score"][0] += 1
            removeFromHand(player, "корабли", "корабли")
            takeCard(deck, sbroses, player)

        elif player["hand"].count("крабы") == 2:
            player["score"][0] += 1
            removeFromHand(player, "крабы", "крабы")

            card = ""
            idx = random.choice([0, 1])
            first_sbros, second_sbros = sbroses[idx], sbroses[1-idx]

            for s in player["strat"]:
                if first_sbros and s in first_sbros:
                    card = s
                    first_sbros.remove(s)
                    break

                if second_sbros and s in second_sbros:
                    card = s
                    second_sbros.remove(s)
                    break

            if card: player["hand"].append(card)

        elif player["hand"].count("акулы") >= 1 and player["hand"].count("пловцы") >= 1:
            player["score"][0] += 1
            removeFromHand(player, "акулы", "пловцы")
            if opponent["hand"]:
                card = random.choice(opponent["hand"])
                opponent["hand"].remove(card)
                player["hand"].append(card)
        else:
            break

def removeFromHand(player, key1, key2):
    player["hand"].remove(key1)
    player["hand"].remove(key2)
    player["played_pair"].append(key1)
    player["played_pair"].append(key2)



def upgradeSetsScore(player):
    score = 0
    for i in range(4):
        cnt = player["hand"].count(NAMES_CARD[i])
        if cnt in SETSPOINTS[i].keys():
            score += SETSPOINTS[i][cnt]
    
    player["score"][1] = score

✅ Основые функции

In [322]:
MY_STRAT = ["ракушки", "крабы", "рыбы", "акулы", "пингвины", "осьминоги", "корабли", "моряки", "пловцы"]

def play_a(player1, player2):
    """
    Раунд завершается, 
    как только один из игроков набрал 7 очков
    """
    resetData(player1)
    resetData(player2)

    deck = genDeck()
    sbroses = [[], []]

    # выбираем кто ходит первый
    players = [player1, player2]
    ind = random.choice([0,1])
    player_1, player_2 = players[ind], players[1-ind]

    # 1 раунд
    while player_1["score"][0] + player_1["score"][1] < 7 and player_2["score"][0] + player_2["score"][1] < 7 and len(deck) > 0:
        takeCard(deck, sbroses, player_1)
        playCard(deck, sbroses, player_1, player_2)
        upgradeSetsScore(player_1)

        # прежде чем дать второму игроку сходить, проверяем, не набрал ли 1 игрок очки
        if player_1["score"][0] + player_1["score"][1] >= 7:
            break
        
        if len(deck) > 0:
            takeCard(deck, sbroses, player_2)
            playCard(deck, sbroses, player_2, player_1)
            upgradeSetsScore(player_2)
    
    score1 = player_1["score"][0] + player_1["score"][1]
    score2 = player_2["score"][0] + player_2["score"][1]

    resetData(player1)
    resetData(player2)
    deck = genDeck()
    sbroses = [[], []]
    
    # 2 раунд
    while player_1["score"][0] + player_1["score"][1] < 7 and player_2["score"][0] + player_2["score"][1] < 7 and len(deck) > 0:
        takeCard(deck, sbroses, player_2)
        playCard(deck, sbroses, player_2, player_1)
        upgradeSetsScore(player_2)

        # прежде чем дать второму игроку сходить, проверяем, не набрал ли 1 игрок очки
        if player_2["score"][0] + player_2["score"][1] >= 7:
            break
        
        if len(deck) > 0:
            takeCard(deck, sbroses, player_1)
            playCard(deck, sbroses, player_1, player_2)
            upgradeSetsScore(player_1)
    
    score1 += player_1["score"][0] + player_1["score"][1]
    score2 += player_2["score"][0] + player_2["score"][1]

    if (player_1["strat"] == MY_STRAT):         
        return score1, score2
    
    return score2, score1

In [323]:
def play_b(player1, player2):
    """
    Раунд завершается, 
    когда колода заканчивается
    """
    resetData(player1)
    resetData(player2)

    deck = genDeck()
    sbroses = [[], []]

    # выбираем кто ходит первый
    players = [player1, player2]
    ind = random.choice([0,1])
    player_1, player_2 = players[ind], players[1-ind]

    turns = {"First round": [0, 0], "Second round": [0, 0]} # ходы[номер раунда] = [ кол-во ходов первый игрок, кол-во ходов второй игрок ]

    # 1 раунд
    while len(deck) > 0:
        if player_1["score"][0] + player_1["score"][1] < 7:
            turns["First round"][0] += 1

        takeCard(deck, sbroses, player_1)
        playCard(deck, sbroses, player_1, player_2)
        upgradeSetsScore(player_1)
        
        if len(deck) > 0:
            if player_2["score"][0] + player_2["score"][1] < 7:
                turns["First round"][1] += 1
            takeCard(deck, sbroses, player_2)
            playCard(deck, sbroses, player_2, player_1)
            upgradeSetsScore(player_2)
    
    score1 = player1["score"][0] + player1["score"][1]
    score2 = player2["score"][0] + player2["score"][1]

    resetData(player1)
    resetData(player2)
    deck = genDeck()
    sbroses = [[], []]

    # 2 раунд
    while len(deck) > 0:
        if player_2["score"][0] + player_2["score"][1] < 7:
            turns["Second round"][1] += 1
        takeCard(deck, sbroses, player_2)
        playCard(deck, sbroses, player_2, player_1)
        upgradeSetsScore(player_2)
        
        if len(deck) > 0:
            if player_1["score"][0] + player_1["score"][1] < 7:
                turns["Second round"][0] += 1
            takeCard(deck, sbroses, player_1)
            playCard(deck, sbroses, player_1, player_2)
            upgradeSetsScore(player_1)
    
    score1 += player1["score"][0] + player1["score"][1]
    score2 += player2["score"][0] + player2["score"][1]

    # в зависимоти от того, какой по счету играла я, вывожу данные
    if (player_1["strat"] == MY_STRAT):
        return score1, score2, turns["First round"][0], turns["Second round"][0]
    
    return score2, score1, turns["First round"][1], turns["Second round"][1]

✅ Генериуем данные и смотрим статистику

In [324]:
# --- стратегии ---
MY_STRAT   = ["ракушки","крабы","рыбы","акулы","пингвины","осьминоги","корабли","моряки","пловцы"]
OP_STRAT1  = ["пловцы","моряки","корабли","осьминоги","пингвины","акулы","рыбы","крабы","ракушки"]
OP_STRAT2  = ["ракушки","корабли","рыбы","акулы","пингвины","осьминоги","крабы","моряки","пловцы"]
OP_STRAT3  = ["осьминоги","крабы","рыбы","акулы","пингвины","ракушки","корабли","моряки","пловцы"]

# --- создаю игроков ---
ME = {"hand": [], "played_pair": [], "strat": MY_STRAT, "score": [0, 0]}
opponent1 = {"hand": [], "played_pair": [], "strat": OP_STRAT1, "score": [0, 0]}
opponent2 = {"hand": [], "played_pair": [], "strat": OP_STRAT2, "score": [0, 0]}
opponent3 = {"hand": [], "played_pair": [], "strat": OP_STRAT3, "score": [0, 0]}

OPPONENTS = {"Reverse Priority": opponent1, "Paired Swap": opponent2, "Unpaired Swap": opponent3}

# --- две функции игры ---
ROUND_FUNCS = {
    "first-to-7": play_a,
    "deck-exhaust": play_b,
}

# --- хранилище результатов раунда ---
# results[стратегия][play a/b] = список пар (очки_меня, очки_оппонента)
results = defaultdict(lambda: defaultdict(list))

turns_list = {strat: [] for strat in OPPONENTS}

def play(me, opp, func, strat_opp, mode):
    """
    me, opp - игроки (я и оппонент)
    func - функция игры из двух раундов (вариант а) или б))
    strat2 - название стратегии оппонента
    mode - название функции ("first-to-7" или "deck-exhaust")
    """
    if func == play_a:
        score_me, score_opp = func(me, opp)
        results[strat_opp][mode].append((score_me, score_opp))
    else:
        score_me, score_opp, turns1, turns2 = func(me, opp)
        results[strat_opp][mode].append((score_me, score_opp))
        turns_list[strat_opp].append(turns1)
        turns_list[strat_opp].append(turns2)

N = 10000
for strat_opp, opp in OPPONENTS.items():
    for play_mode, play_func in ROUND_FUNCS.items():
        for i in range(N):
            play(ME, opp, play_func, strat_opp, play_mode)


# --- сводим статистику ---
summary = {}
for strategy, play_modes in results.items():
    summary[strategy] = {}
    for mode, scores in play_modes.items():
        arr = np.array(scores)
        avg_me = arr[:,0].mean()
        avg_op = arr[:,1].mean()
        win_pct = np.mean(arr[:,0] > arr[:,1]) * 100
        summary[strategy][mode] = {
            "avg_me": round(avg_me,2),
            "avg_op": round(avg_op,2),
            "win_%":  round(win_pct,2)
        }

# распечатаем:
for strategy, modes in summary.items():
    print(f"Стратегия оппонента: {strategy}")
    print(f"Mode\t\tAvg me\t\tAvg opponent\tWin %")
    for mode, stats in modes.items():
        print(f"{mode}\t{stats['avg_me']}\t\t{stats['avg_op']}\t\t{stats['win_%']}%")
    print()



Стратегия оппонента: Reverse Priority
Mode		Avg me		Avg opponent	Win %
first-to-7	12.27		10.95		55.97%
deck-exhaust	33.02		33.13		47.29%

Стратегия оппонента: Paired Swap
Mode		Avg me		Avg opponent	Win %
first-to-7	11.35		12.45		36.26%
deck-exhaust	24.71		24.73		46.97%

Стратегия оппонента: Unpaired Swap
Mode		Avg me		Avg opponent	Win %
first-to-7	10.99		12.49		35.15%
deck-exhaust	29.34		29.34		47.71%



## **Задание 4:**

**С помощью подходящего критерия для каждого соперника проверьте гипотезу о том, что стратегии одинаково успешны друг против друга, при условии, что набрав 7 очков, любой игрок моментально будет завершать раунд.**

$$H_0: p_{win} = 0$$

В каждой игре есть только два исхода (мы не считаем ничьи): либо ты выиграл, либо проиграл.
Это похоже на биномиальную модель.

Z-тест для доли это нормальное приближение биномиального распределения при больших $n$, а также он требует, чтобы было достаточно “успехов” и “не-успехов”; у нас эти условия точно выполняются, поэтому выберу его в качестве критерия.


- n — число "решённых" игр (без ничьих),
- x — число моих побед,
- $\hat{p} = \frac{x}{n}$ — наблюдаемая доля побед,
- $p_0 = 0.5$ — доля по $H_0.$


In [325]:
alpha = 0.05
p0 = 0.5

for strategy, play_modes in results.items():
    scores = play_modes["first-to-7"]

    wins   = sum(1 for me, opp in scores if me >  opp)
    losses = sum(1 for me, opp in scores if me <  opp)
    ties   = sum(1 for me, opp in scores if me == opp)

    n = wins + losses

    z_stat, p_val = proportions_ztest(wins, n, p0, alternative='two-sided')

    print(f"z = {z_stat:.3f}, p-value = {p_val:.4e}")
    if p_val < alpha:
        print("ОТВЕРГАЕМ H₀: стратегии различаются")
    else:
        print("НЕ отвергаем H₀: статистически не различаются")
    
    print("")


z = 21.146, p-value = 3.0329e-99
ОТВЕРГАЕМ H₀: стратегии различаются

z = -19.204, p-value = 3.3965e-82
ОТВЕРГАЕМ H₀: стратегии различаются

z = -23.131, p-value = 2.2355e-118
ОТВЕРГАЕМ H₀: стратегии различаются



## **Задание 5:**

**С помощью подходящего критерия для каждого соперника проверьте гипотезу о том, что стратегии одинаково успешны друг против друга, при условии, что оба игрока не хотят завершать раунд и будут играть, пока не закончится колода.**

In [326]:
for strategy, play_modes in results.items():
    scores = play_modes["deck-exhaust"]

    wins   = sum(1 for me, opp in scores if me >  opp)
    losses = sum(1 for me, opp in scores if me <  opp)
    ties   = sum(1 for me, opp in scores if me == opp)

    n = wins + losses

    z_stat, p_val = proportions_ztest(wins, n, p0, alternative='two-sided')

    print(f"z = {z_stat:.3f}, p-value = {p_val:.4e}")
    if p_val < alpha:
        print("ОТВЕРГАЕМ H₀: стратегии различаются")
    else:
        print("НЕ отвергаем H₀: статистически не различаются")
    
    print("")

z = -1.267, p-value = 2.0520e-01
НЕ отвергаем H₀: статистически не различаются

z = -0.350, p-value = 7.2622e-01
НЕ отвергаем H₀: статистически не различаются

z = 0.092, p-value = 9.2656e-01
НЕ отвергаем H₀: статистически не различаются



# **Задание 6:**

**С помощью бутстрапа постройте доверительный интервал для медианы количества ходов, за которые ваша стратегия набирала 7 очков в партиях из пункта 5 (или длительность раунда, если так и не набрали 7 очков за весь раунд).**

In [328]:

def bootstrap_median(data, B=10000, alpha=0.05):
    n = len(data)
    medians = np.empty(B)
    # мы получим B значений медиан «подвыборок».
    for b in range(B):
        sample = np.random.choice(data, size=n, replace=True) # Из data случайно выбираются n элементов с возвращением 
        medians[b] = np.median(sample)
    lo = np.percentile(medians, 100 * (alpha/2))
    hi = np.percentile(medians, 100 * (1 - alpha/2))
    return np.median(data), lo, hi

for strat_name, data in turns_list.items():
    median, lo, hi = bootstrap_median(data, B=10000, alpha=0.05)
    print(f"{strat_name:>12}: медиана ходов = {median:.1f}, 95% CI = [{lo:.1f}, {hi:.1f}]")

Reverse Priority: медиана ходов = 10.0, 95% CI = [10.0, 10.0]
 Paired Swap: медиана ходов = 10.0, 95% CI = [10.0, 10.0]
Unpaired Swap: медиана ходов = 9.0, 95% CI = [9.0, 9.0]
